In [1]:
import numpy as np
import time
#Defining Grid

#defining action variables
up=0
down=1
left=2
right=3

#no. of states and no. of variables
noS=4*4
noA=4

S=range(noS)

reward =-1    #for every step

terminal_state = lambda s: s==0 or s==noS-1  #first and last state terminal
wall=[]

P=dict() #transition probabilities

for s in S:
    P[s]=dict()
    
    if (terminal_state(s)):
        P[s][up]=(s,1.0,0.0)   # next_state, probability, reward
        P[s][down]=(s,1.0,0.0)
        P[s][right]=(s,1.0,0.0)
        P[s][left]=(s,1.0,0.0)
    else:
        next_s= s if(s<4) else s-4
        if next_s in wall:
            P[s][up]=(s,1.0,-1000.0)
        else:
            P[s][up]=(next_s,1.0,reward)
        
        next_s= s if(16-s<=4) else s+4
        if next_s in wall:
            P[s][down]=(s,1.0,-1000.0)
        else:
            P[s][down]=(next_s,1.0,reward)
        
        next_s= s if((s+1)%4==0) else s+1
        if next_s in wall:
            P[s][right]=(s,1.0,-1000.0)
        else:
            P[s][right]=(next_s,1.0,reward)
        
        next_s= s if(s%4==0) else s-1
        if next_s in wall:
            P[s][left]=(s,1.0,-1000.0)
        else:
            P[s][left]=(next_s,1.0,reward)
        
        
print 'No. of states in grid: ', noS
print 'No. of action options in each state:', noA
Action_Index=dict()
Action_Index[0]='up'
Action_Index[1]='down'
Action_Index[2]='left'
Action_Index[3]='right'
Action_Index[5]='terminal states (stay)'
Action_Index[7]='wall'

print 'Index for actions:'
for k,v in Action_Index.items():
    print k,":",v

No. of states in grid:  16
No. of action options in each state: 4
Index for actions:
0 : up
1 : down
2 : left
3 : right
5 : terminal states (stay)
7 : wall


In [2]:
#Policy Evaluation

def policy_evaluation(P,policy,threshold,dicount):
    value=np.zeros((noS,))
    while True:
        new_value=np.zeros((noS,))

        change=0
        for s in S:
            v=0
            
            for a,action_prob in enumerate(policy[s]):
                next_state,probability,reward=P[s][a]
                temp=probability*action_prob*(reward+discount*value[next_state])
                v+=temp

            change=max(change,np.abs(v-value[s]))
            new_value[s]=v

        if change < threshold:
              break

        value=new_value

    return value

In [3]:
#Testing policy evaluation 
random_policy = np.ones([16, 4])/4

threshold = 0.0001
discount = 1.0
value = np.zeros((noS,))

random_policy_value=policy_evaluation(P,random_policy,threshold,discount)
random_policy_value[wall]=13
print 'Value Function for policy: all actions equiprobable:'
print random_policy_value.reshape(4,4)

Value Function for policy: all actions equiprobable:
[[  0.         -13.99887902 -19.99833891 -21.99814115]
 [-13.99887902 -17.99853668 -19.99835003 -19.99833891]
 [-19.99833891 -19.99835003 -17.99853668 -13.99887902]
 [-21.99814115 -19.99833891 -13.99887902   0.        ]]


In [4]:
#Policy Iteration

def policy_iteration(P,discount,threshold):
    #Initialisation
    value=np.zeros((noS,))
    policy=np.ones([16, 4])/4
    
    while True:
        #Policy evaluation
        value=policy_evaluation(P,policy,threshold,discount)
        
        new_value=np.zeros((noS,))
        new_policy=np.zeros([noS,4])
        
        #Policy Improvement
        policy_stable=True
        for s in S:
            if s!=0 and s!=15:
                old_action=policy[s]
                action_values = np.zeros(noA)
                for a in range(noA):   		# Iterating over all the actions     
                        next_state,probability,reward = P[s][a]
                        action_values[a] += probability*(reward + discount*value[next_state])
                max_total = np.amax(action_values)   # taking the max reward value 
                best_a = np.argmax(action_values)

                new_policy[s][best_a]=1

                new_value[s]=max_total 
                if (np.array_equal(old_action,new_policy[s])!=True):
                    policy_stable=False
                    
        value=new_value
        
        if policy_stable:
            value[wall]=13
            return new_policy,value
        else:
            policy=new_policy 

start=time.clock()
best_policy,corr_value=policy_iteration(P,discount,threshold)
end=time.clock()

show_best_policy=np.zeros(noS,)
for s,p_s in enumerate(best_policy):
    if terminal_state(s):
        show_best_policy[s]=5
    elif s in wall:
        show_best_policy[s]=7
    else:
        show_best_policy[s]=np.argmax(p_s)

    
print 'Best policy with Policy Iteration is'
print show_best_policy.reshape(4,4)
print 'Corresponding Value Function is'
print corr_value.reshape(4,4)
print 'Time taken'
print end-start

Best policy with Policy Iteration is
[[ 5.  2.  2.  1.]
 [ 0.  0.  0.  1.]
 [ 0.  0.  1.  1.]
 [ 0.  3.  3.  5.]]
Corresponding Value Function is
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
Time taken
0.0619498827247


In [1]:
#Value Iteration

def value_iteration(P,discount,threshold):
    #Initialisation
    value=np.zeros((noS,))
    
    while True:
        new_policy=np.zeros([noS,4])
        change=0
        for s in S:
            if s!=0 and s!=15:
                v=value[s]
                action_values = np.zeros(noA)
                for a in range(noA):   		# Iterating over all the actions     
                        next_state,probability,reward = P[s][a]
                        action_values[a] += probability*(reward + discount*value[next_state])
                max_total = np.amax(action_values)   # taking the max reward value 
                best_a = np.argmax(action_values)

                value[s]=max_total
                new_policy[s][best_a]=1

                change=max(change,np.abs(v-value[s]))
            
        if change < threshold:
              break
    
    value[wall]=13            
    return new_policy,value.reshape(4,4)

start=time.clock()
best_policy,corr_value=value_iteration(P,discount,threshold)
end=time.clock()

show_best_policy=np.zeros(noS,)
for s,p_s in enumerate(best_policy):
    if terminal_state(s):
        show_best_policy[s]=5
    elif s in wall:
        show_best_policy[s]=7
    else:
        show_best_policy[s]=np.argmax(p_s)
    
print 'Best policy with Value Iteration is'
print show_best_policy.reshape(4,4)
print 'Corresponding Value Function is'
print corr_value.reshape(4,4)
print 'Time taken'
print end-start

NameError: name 'time' is not defined

In [6]:
print 'Our Value Function:'
print corr_value.reshape(4,4)

Our Value Function:
[[ 0. -1. -2. -3.]
 [-1. -2. -3. -2.]
 [-2. -3. -2. -1.]
 [-3. -2. -1.  0.]]
